# Recurrent Neural Networks (RNN)

In [2]:
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import plotly.express as px

# Train test split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Word2Vec Embedding
import gensim
from gensim.models import Word2Vec
from sklearn.utils import resample


from gensim.models import FastText
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


from keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ELMo Embedding
import tensorflow as tf
import tensorflow_hub as hub
import h5py

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator

#RNN
from tensorflow import keras
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier 
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Flatten

## Load cleaned dataset

In [3]:
# Read from CSV file
df = pd.read_csv('cleaned_combined_data.csv')
df.head()

,Categorization,Body,Label,Cleaned Text,Cleaned Text with N lemmatization,Cleaned Text with V lemmatization,Cleaned Text with A lemmatization
0,Envy to other is swallowing me,"Im from developingcountry, Indonesia , and for...",1,im developingcountry indonesia temporary work ...,im developingcountry indonesia temporary work ...,im developingcountry indonesia temporary work ...,im developingcountry indonesia temporary work ...
1,Nothin outta the ordinary. Paradise. Job stres...,Um hello ....well many can relate im sure. Aft...,1,um hello well many relate im sure today im con...,um hello well many relate im sure today im con...,um hello well many relate im sure today im con...,um hello well many relate im sure today im con...
2,Almost 49 and the chasm of emptiness has never...,I’ve been diagnosed severe bi polar where you ...,1,ive diagnosed severe bi polar longer even get ...,ive diagnosed severe bi polar longer even get ...,ive diagnose severe bi polar longer even get g...,ive diagnosed severe bi polar long even get go...
3,I’m happy again,"After my closest friend left me in April, I ha...",0,closest friend left april finally let go reali...,closest friend left april finally let go reali...,closest friend leave april finally let go real...,close friend left april finally let go realize...
4,Is it possible to recover from such a traumati...,"I am only 15, and yet I feel my life is alread...",1,15 yet feel life already pit emptiness stomach...,15 yet feel life already pit emptiness stomach...,15 yet feel life already pit emptiness stomach...,15 yet feel life already pit emptiness stomach...


In [4]:
df.dropna(inplace=True)

## Hypothesis 1: How does different text lemmatization affect the model results?

In [5]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, make_scorer
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from sklearn.model_selection import RandomizedSearchCV

In [6]:
def create_rnn_model(embedding_dim, num_units, learning_rate, vocab_size, max_sequence_length):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
    model.add(SimpleRNN(units=num_units, activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model


def hypothesis1_test(column_name, perform_hyperparameter_tuning=False):
    # Initialize f1_score and recall
    f1 = 0.0
    recall = 0.0
    
    # Split the data into training and testing sets
    X = df[column_name]
    y = df['Label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Tokenize the text
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)

    # Define the vocabulary size
    vocab_size = len(tokenizer.word_index) + 1

    # Set the maximum sequence length (adjust as needed)
    max_sequence_length = 100  # You can choose an appropriate sequence length

    if perform_hyperparameter_tuning:
        # Define the hyperparameter distributions for RandomizedSearchCV
        param_dist = {
            'embedding_dim': np.arange(50, 151, 10),  # Range of values for embedding_dim
            'num_units': np.arange(32, 193, 32),      # Range of values for num_units
            'learning_rate': [1e-2, 1e-3, 1e-4]
        }

        # Create a custom scorer for F1-score using the f1_score function
        recall_scorer = make_scorer(recall_score)

        # Create the RandomizedSearchCV
        rnn_model = KerasClassifier(model=create_rnn_model, vocab_size=vocab_size, max_sequence_length=max_sequence_length, num_units=32, learning_rate=0.01, embedding_dim=50)
        random_search = RandomizedSearchCV(estimator=rnn_model, param_distributions=param_dist, scoring=recall_scorer, cv=5, n_iter=10, verbose=1)

        # Convert text to sequences and pad them to the specified length
        X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_sequence_length)

        # Fit the random search to the training data with numerical features
        random_search.fit(X_train_seq, y_train)

        best_hyperparameters = random_search.best_params_
        # Create the best model with the best hyperparameters
        best_model = create_rnn_model(embedding_dim=best_hyperparameters['embedding_dim'],
                                      num_units=best_hyperparameters['num_units'],
                                      learning_rate=best_hyperparameters['learning_rate'],
                                      vocab_size=vocab_size,
                                      max_sequence_length=max_sequence_length)

        # Compile the best model
        best_model.compile(optimizer=keras.optimizers.Adam(learning_rate=best_hyperparameters['learning_rate']),
                          loss='binary_crossentropy',
                          metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

        # Use the best model to make predictions on the test set
        X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_sequence_length)
        y_test_pred = (best_model.predict(X_test_seq) > 0.5).astype(int)

        # Calculate classification error
        accuracy = accuracy_score(y_test, y_test_pred)
        classification_error = 1 - accuracy

        # Calculate recall, specificity, F1 score, and AUC score
        tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
        recall = tp / (tp + fn)
        specificity = tn / (tn + fp)
        precision = precision_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred)

        auc_score = roc_auc_score(y_test, y_test_pred)

        print(f"Test accuracy: {accuracy * 100:.2f}%")
        print(f"Classification Error: {classification_error * 100:.2f}%")
        print(f"Recall: {recall * 100:.2f}%")
        print(f"Specificity: {specificity * 100:.2f}%")
        print(f"Precision: {precision * 100:.2f}%")
        print(f"F1 Score: {f1 * 100:.2f}%")
        print(f"AUC Score: {auc_score * 100:.2f}%")

        print("Best Hyperparameters:")
        print(f"Embedding Dim: {best_hyperparameters.get('embedding_dim')}")
        print(f"Num Units: {best_hyperparameters.get('num_units')}")
        print(f"Learning Rate: {best_hyperparameters.get('learning_rate')}")

    else:

        # Convert text to sequences and pad them to the specified length
        X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_sequence_length)
        X_test = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_sequence_length)

        embedding_dim = 100  # Set your desired embedding dimension

        model = create_rnn_model(embedding_dim=embedding_dim, num_units=32, learning_rate=0.01, vocab_size=vocab_size, max_sequence_length=max_sequence_length)

        # Compile the model
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='binary_crossentropy',
                      metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

        batch_size = 32  # Set your desired batch size
        num_epochs = 10  # Set the number of training epochs

        model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs)

        result = model.evaluate(X_test, y_test)
        loss = result[0]
        accuracy = result[1]

        # Use the model to make predictions on the test set
        y_pred = model.predict(X_test)
        y_pred = (y_pred > 0.5).astype(int)  # Convert predicted probabilities to binary labels

        # Calculate classification error
        classification_error = 1 - accuracy

        # Calculate recall, specificity, F1 score, and AUC score
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        recall = tp / (tp + fn)
        specificity = tn / (tn + fp)
        precision = precision_score(y_test, y_pred)
        f1 = 2 * (precision * recall) / (precision + recall)
        auc_score = roc_auc_score(y_test, y_pred)

        print(f"Test accuracy: {accuracy * 100:.2f}%")
        print(f"Classification Error: {classification_error * 100:.2f}%")
        print(f"Recall: {recall * 100:.2f}%")
        print(f"Specificity: {specificity * 100:.2f}%")
        print(f"Precision: {precision * 100:.2f}%")
        print(f"F1 Score: {f1 * 100:.2f}%")
        print(f"AUC Score: {auc_score * 100:.2f}%")

        print(classification_report(y_test, y_pred))  # This


In [7]:
hypothesis1_test('Cleaned Text', perform_hyperparameter_tuning=False)

Epoch 1/10
360/360 [==============================] - 9s 23ms/step - loss: 0.5548 - accuracy: 0.7280 - precision_1: 0.7426 - recall_1: 0.8340
Epoch 2/10
360/360 [==============================] - 8s 23ms/step - loss: 0.3793 - accuracy: 0.8424 - precision_1: 0.8571 - recall_1: 0.8835
Epoch 3/10
360/360 [==============================] - 8s 22ms/step - loss: 0.2927 - accuracy: 0.8810 - precision_1: 0.8880 - recall_1: 0.9165
Epoch 4/10
360/360 [==============================] - 8s 22ms/step - loss: 0.2381 - accuracy: 0.9084 - precision_1: 0.9175 - recall_1: 0.9302
Epoch 5/10
360/360 [==============================] - 8s 22ms/step - loss: 0.2098 - accuracy: 0.9231 - precision_1: 0.9311 - recall_1: 0.9409
Epoch 6/10
360/360 [==============================] - 8s 22ms/step - loss: 0.1847 - accuracy: 0.9343 - precision_1: 0.9407 - recall_1: 0.9499
Epoch 7/10
360/360 [==============================] - 8s 22ms/step - loss: 0.1627 - accuracy: 0.9415 - precision_1: 0.9450 - recall_1: 0.9579
Epoch 

In [8]:
hypothesis1_test('Cleaned Text with N lemmatization', perform_hyperparameter_tuning=False)

Epoch 1/10
360/360 [==============================] - 9s 24ms/step - loss: 0.5532 - accuracy: 0.7276 - precision_3: 0.7377 - recall_3: 0.8445
Epoch 2/10
360/360 [==============================] - 9s 24ms/step - loss: 0.3817 - accuracy: 0.8395 - precision_3: 0.8530 - recall_3: 0.8838
Epoch 3/10
360/360 [==============================] - 9s 24ms/step - loss: 0.3243 - accuracy: 0.8717 - precision_3: 0.8793 - recall_3: 0.9103
Epoch 4/10
360/360 [==============================] - 9s 24ms/step - loss: 0.3096 - accuracy: 0.8773 - precision_3: 0.8944 - recall_3: 0.9010
Epoch 5/10
360/360 [==============================] - 9s 24ms/step - loss: 0.2638 - accuracy: 0.8948 - precision_3: 0.9103 - recall_3: 0.9139
Epoch 6/10
360/360 [==============================] - 9s 24ms/step - loss: 0.2194 - accuracy: 0.9151 - precision_3: 0.9250 - recall_3: 0.9336
Epoch 7/10
360/360 [==============================] - 9s 24ms/step - loss: 0.1812 - accuracy: 0.9314 - precision_3: 0.9421 - recall_3: 0.9432
Epoch 

In [9]:
hypothesis1_test('Cleaned Text with V lemmatization', perform_hyperparameter_tuning=False)

Epoch 1/10
360/360 [==============================] - 9s 22ms/step - loss: 0.5538 - accuracy: 0.7325 - precision_5: 0.7355 - recall_5: 0.8624
Epoch 2/10
360/360 [==============================] - 8s 22ms/step - loss: 0.4344 - accuracy: 0.8107 - precision_5: 0.8173 - recall_5: 0.8798
Epoch 3/10
360/360 [==============================] - 8s 24ms/step - loss: 0.3684 - accuracy: 0.8435 - precision_5: 0.8517 - recall_5: 0.8938
Epoch 4/10
360/360 [==============================] - 8s 23ms/step - loss: 0.3273 - accuracy: 0.8698 - precision_5: 0.8785 - recall_5: 0.9077
Epoch 5/10
360/360 [==============================] - 7s 20ms/step - loss: 0.2967 - accuracy: 0.8842 - precision_5: 0.8903 - recall_5: 0.9195
Epoch 6/10
360/360 [==============================] - 7s 19ms/step - loss: 0.2774 - accuracy: 0.8896 - precision_5: 0.8992 - recall_5: 0.9181
Epoch 7/10
360/360 [==============================] - 7s 19ms/step - loss: 0.2416 - accuracy: 0.9082 - precision_5: 0.9130 - recall_5: 0.9355
Epoch 

In [10]:
hypothesis1_test('Cleaned Text with A lemmatization', perform_hyperparameter_tuning=False)

Epoch 1/10
360/360 [==============================] - 9s 22ms/step - loss: 0.5397 - accuracy: 0.7365 - precision_7: 0.7557 - recall_7: 0.8261
Epoch 2/10
360/360 [==============================] - 8s 22ms/step - loss: 0.3577 - accuracy: 0.8559 - precision_7: 0.8665 - recall_7: 0.8970
Epoch 3/10
360/360 [==============================] - 8s 22ms/step - loss: 0.2819 - accuracy: 0.8915 - precision_7: 0.8998 - recall_7: 0.9209
Epoch 4/10
360/360 [==============================] - 8s 22ms/step - loss: 0.2161 - accuracy: 0.9159 - precision_7: 0.9231 - recall_7: 0.9374
Epoch 5/10
360/360 [==============================] - 8s 22ms/step - loss: 0.1759 - accuracy: 0.9358 - precision_7: 0.9410 - recall_7: 0.9522
Epoch 6/10
360/360 [==============================] - 8s 22ms/step - loss: 0.1477 - accuracy: 0.9467 - precision_7: 0.9506 - recall_7: 0.9607
Epoch 7/10
360/360 [==============================] - 8s 22ms/step - loss: 0.1457 - accuracy: 0.9451 - precision_7: 0.9524 - recall_7: 0.9559
Epoch 

In [11]:
hypothesis1_test('Cleaned Text with A lemmatization', perform_hyperparameter_tuning=True)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
90/90 [==============================] - 1s 7ms/step
Test accuracy: 52.70%
Classification Error: 47.30%
Recall: 69.20%
Specificity: 30.15%
Precision: 57.52%
F1 Score: 62.82%
AUC Score: 49.67%
Best Hyperparameters:
Embedding Dim: 110
Num Units: 192
Learning Rate: 0.0001
